<a href="https://colab.research.google.com/github/mberkancetin/fuzzy-ahp-color/blob/main/test/multiAHPy_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/mberkancetin/fuzzy-ahp-color.git
import sys
sys.path.insert(0, '/content/fuzzy-ahp-color')

# **PART 1: FUZZY AHP DEMONSTRATION (COLOR Case)**

## Step 1: Building the Hierarchy

In [ ]:
import numpy as np
from multiAHPy.model import Node, Alternative, Hierarchy
from multiAHPy.types import TFN, TrFN, Crisp, GFN, NumericType, Number
from multiAHPy.validation import Validation
from multiAHPy.consistency import Consistency

goal = Node("Goal", "Corporate Local Responsibility (COLOR) Score")
criteria_nodes = {
    "C1": Node("C1", "Generosity & Benevolence"),
    "C2": Node("C2", "Societal Demand"),
    "C3": Node("C3", "Compliance"),
    "C4": Node("C4", "Stakeholder Involvement")
}
for c_node in criteria_nodes.values():
    goal.add_child(c_node)

sub_criteria_data = {
    "C1": ["C11", "C12", "C13", "C14"],
    "C2": ["C21", "C22", "C23", "C24"],
    "C3": ["C31", "C32", "C33", "C34"],
    "C4": ["C41", "C42", "C43", "C44"]
}
for parent_id, child_ids in sub_criteria_data.items():
    for child_id in child_ids:
        criteria_nodes[parent_id].add_child(Node(child_id))

In [ ]:
goal.__repr__()

In [ ]:
goal.children

In [ ]:
goal.children[0].children

## Step 2: Instantiate the Model for FUZZY (TFN) operations ---
    We explicitly tell the model it will be working with TFNs.

In [ ]:
fuzzy_model = Hierarchy[TFN](root_node=goal, number_type=TFN)
fuzzy_model.display()

## Step 3: Set FUZZY Comparison Matrices ---
    In a real scenario, these would come from your surveys and be TFNs.
    We will create mock TFN matrices for this demonstration.

In [ ]:
def create_mock_tfn_matrix(size):
    """
    Creates a mock TFN matrix that correctly follows the reciprocity rule.
    """
    matrix = np.empty((size, size), dtype=object)
    for i in range(size):
        for j in range(size):
            if i == j:
                # Diagonal elements are always (1, 1, 1)
                matrix[i, j] = TFN(1, 1, 1)
            elif i < j: # Only fill the upper triangle with random values
                val = np.random.uniform(1, 9)
                matrix[i, j] = TFN(max(1, val-1), val, val+1)
            else: # For the lower triangle, do nothing yet
                continue

    # Now, fill the lower triangle with the inverses of the upper triangle
    for i in range(size):
        for j in range(i + 1, size):
            matrix[j, i] = matrix[i, j].inverse()

    return matrix

fuzzy_model.set_comparison_matrix("Goal", create_mock_tfn_matrix(4))
fuzzy_model.set_comparison_matrix("C1", create_mock_tfn_matrix(4))
fuzzy_model.set_comparison_matrix("C2", create_mock_tfn_matrix(4))
fuzzy_model.set_comparison_matrix("C3", create_mock_tfn_matrix(4))
fuzzy_model.set_comparison_matrix("C4", create_mock_tfn_matrix(4))


In [ ]:
fuzzy_model.root.children

## Step 4: Check Matrix Consistencies

In [ ]:
# Call the new, cleaner method
consistency_results = fuzzy_model.check_consistency(threshold=0.1)

is_fully_consistent = True
# The structure of the results dictionary is the same, so this loop works as is
for node_id, result in consistency_results.items():
    print(f"  - Consistency for '{node_id}' matrix: CR = {result['consistency_ratio']:.4f} "
        f"(Is Consistent: {result['is_consistent']})")
    if not result['is_consistent']:
        is_fully_consistent = False

if not is_fully_consistent:
    print("\nWARNING: One or more matrices are inconsistent...")
    # ... the rest of your error handling and recommendation logic ...
else:
    print("All matrices are consistent. Proceeding with calculations.")
print(Consistency.get_consistency_recommendations(fuzzy_model, "C1"))

## Step 5: Calculate Weights

In [ ]:
fuzzy_model.calculate_weights(method="geometric_mean")

## Step 6: Add Alternatives with FUZZY Performance Scores

In [ ]:
company_a_fuzzy = Alternative("Company A")

# Performance can also be uncertain. A score of 0.78 might be TFN(0.7, 0.78, 0.85)
leaf_nodes = fuzzy_model.root.get_all_leaf_nodes()
for leaf in leaf_nodes:
    # Create random fuzzy scores for the demo
    m = np.random.uniform(0.5, 0.9)
    l, u = max(0, m-0.1), min(1, m+0.1)
    company_a_fuzzy.set_performance_score(leaf.id, TFN(l, m, u))

company_b_fuzzy = Alternative("Company B")
for leaf in leaf_nodes:
    m = np.random.uniform(0.4, 0.8)
    l, u = max(0, m-0.1), min(1, m+0.1)
    company_b_fuzzy.set_performance_score(leaf.id, TFN(l, m, u))

fuzzy_model.add_alternative(company_a_fuzzy)
fuzzy_model.add_alternative(company_b_fuzzy)

## Step 7: Validating the Model Setup

In [ ]:
validation_results = Validation.run_all_validations(fuzzy_model)

# Check if there are any errors
has_errors = any(len(error_list) > 0 for error_list in validation_results.values())

if has_errors:
    print("VALIDATION FAILED. Please fix the following issues:")
    for category, errors in validation_results.items():
        if errors:
            print(f"\n  Category: {category}")
            for error in errors:
                print(f"    - {error}")

## Step 8: Run the Final Score Calculation

In [ ]:
fuzzy_model.calculate_weights()
fuzzy_model.calculate_alternative_scores()


print("\n\n--- FUZZY AHP FINAL RESULTS ---")
# The overall scores are TFNs, so we must defuzzify to rank.
print("Overall Fuzzy Scores:")
for alt in fuzzy_model.alternatives:
    print(f"  - {alt.name}: {alt.overall_score}")

print("\nFinal Crisp Rankings (using 'centroid' defuzzification):")
fuzzy_rankings = fuzzy_model.get_rankings(defuzzify_method='centroid')
for i, (name, score) in enumerate(fuzzy_rankings):
    print(f"{i+1}. {name}: {score:.4f}")

fuzzy_model.display()

## Step 9: Generating Full Analysis Report

In [ ]:
report_string = fuzzy_model.full_report()
print(report_string)

## Step 10: Exporting Full Analysis Report to Google Sheets

In [ ]:
# First, make sure to install the necessary libraries in a Colab cell
!pip install gspread gspread-dataframe google-auth pandas openpyxl

spreadsheet_id is required:
1. Go to Google Sheets. here: [sheets.google.com](sheets.google.com)  
2. Create a new sheet.
3. Copy the ID from the URL.
4. Paste it into the code.

After giving permission to write in the given spreadsheet, following will be printed (spreadsheet id is an example, please provide spreadsheet id below)

Starting Google Sheets export...
  - Authenticating with Google...
  - Authentication successful.
  - Successfully opened existing spreadsheet: 'myFAHanalysis'
    - Uploaded report for 'Goal' to sheet 'Goal'.
    - Uploaded report for 'C1' to sheet 'C1'.
    - Uploaded report for 'C2' to sheet 'C2'.
    - Uploaded report for 'C3' to sheet 'C3'.
    - Uploaded report for 'C4' to sheet 'C4'.

✅ Google Sheets report complete!
   URL: https://docs.google.com/spreadsheets/d/PleaSe3nterY0ur-Spr34dshEetID

In [ ]:
spreadsheet_id = "PleaSe3nterY0ur-Spr34dshEetID" #@param {type:"string"}

In [ ]:
fuzzy_model.export_report(target="myFAHanalysis", spreadsheet_id=spreadsheet_id, output_format='gsheet')

# **PART 2: CLASSIC AHP DEMONSTRATION**

First, comparison with pyDecision to be sure results are same.

In [ ]:
!git clone https://github.com/Valdecy/pyDecision.git
import sys
sys.path.insert(0, '/content/pyDecision')

import numpy as np
from pyDecision.algorithm import ahp_method

# Parameters
weight_derivation = 'max_eigen' # 'mean'; 'geometric' or 'max_eigen'

# Dataset
dataset = np.array([
  #g1     g2     g3     g4     g5     g6     g7
  [1  ,   1/3,   1/5,   1  ,   1/4,   1/2,   3  ],   #g1
  [3  ,   1  ,   1/2,   2  ,   1/3,   3  ,   3  ],   #g2
  [5  ,   2  ,   1  ,   4  ,   5  ,   6  ,   5  ],   #g3
  [1  ,   1/2,   1/4,   1  ,   1/4,   1  ,   2  ],   #g4
  [4  ,   3  ,   1/5,   4  ,   1  ,   3  ,   2  ],   #g5
  [2  ,   1/3,   1/6,   1  ,   1/3,   1  ,   1/3],   #g6
  [1/3,   1/3,   1/5,   1/2,   1/2,   3  ,   1  ]    #g7
])

# Call AHP Function
weights, rc = ahp_method(dataset, wd = weight_derivation)

# Weigths
for i in range(0, weights.shape[0]):
  print('w(g'+str(i+1)+'): ', round(weights[i], 3))

# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

In [ ]:
!git clone https://github.com/mberkancetin/fuzzy-ahp-color.git
import sys
sys.path.insert(0, '/content/fuzzy-ahp-color')

from multiAHPy.model import Node, Alternative, Hierarchy
from multiAHPy.types import TFN, TrFN, Crisp, GFN, NumericType, Number
from multiAHPy.validation import Validation
from multiAHPy.consistency import Consistency

goal = Node("Goal", "Corporate Local Responsibility (COLOR) Score")
criteria_nodes = {
    "C1": Node("C1", "Generosity & Benevolence"),
    "C2": Node("C2", "Societal Demand"),
    "C3": Node("C3", "Compliance"),
    "C4": Node("C4", "Stakeholder Involvement"),
    "C5": Node("C5", "Environment"),
    "C6": Node("C6", "Social"),
    "C7": Node("C7", "Governance")
}
for c_node in criteria_nodes.values():
    goal.add_child(c_node)

crisp_model = Hierarchy[Crisp](root_node=goal, number_type=Crisp)

crisp_model.set_comparison_matrix("Goal", dataset)
crisp_model.calculate_weights(method="eigenvector")

In [ ]:
crisp_model.display()

## Final Step: Add Alternatives with CRISP Performance Scores

In [ ]:
company_a_crisp = Alternative("Company A")
leaf_nodes_crisp = crisp_model.root.get_all_leaf_nodes()
for leaf in leaf_nodes_crisp:
    # Scores are simple floats wrapped in the Crisp class
    company_a_crisp.set_performance_score(leaf.id, Crisp(np.random.uniform(0.5, 0.9)))

company_b_crisp = Alternative("Company B")
for leaf in leaf_nodes_crisp:
    company_b_crisp.set_performance_score(leaf.id, Crisp(np.random.uniform(0.4, 0.8)))

crisp_model.add_alternative(company_a_crisp)
crisp_model.add_alternative(company_b_crisp)
print("Alternatives with Crisp scores added.")

# --- Calculate Final Crisp Scores and Rank ---
crisp_model.calculate_alternative_scores()

print("\n\n--- CLASSIC AHP FINAL RESULTS ---")
# The overall scores are already crisp, but get_rankings still works.
print("Overall Crisp Scores:")
for alt in crisp_model.alternatives:
    print(f"  - {alt.name}: {alt.overall_score}")

print("\nFinal Crisp Rankings:")
crisp_rankings = crisp_model.get_rankings()
for i, (name, score) in enumerate(crisp_rankings):
    print(f"{i+1}. {name}: {score:.4f}")

In [ ]:
crisp_model.display()

In [ ]:
crisp_model.summary("Company A")

In [ ]:
crisp_model.summary("Company B")